In [1]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers import Conv2D, MaxPooling2D

Using TensorFlow backend.


In [17]:
import os
os.getcwd()
import numpy as np
import pandas as pd

In [5]:
test = np.genfromtxt('test.csv', delimiter=",")
train = np.genfromtxt('train.csv', delimiter=',')

In [6]:
train.shape

(42001, 785)

In [7]:
test.shape

(28001, 784)

In [18]:
trainp = pd.read_csv('train.csv')

In [41]:
testp = pd.read_csv('test.csv')

In [21]:
# convert the label to categorical vector and store it in y_train
from keras.utils import np_utils
y_train = np_utils.to_categorical(trainp.label)

In [23]:
y_train[0]

array([ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])

In [27]:
# checking if there are any NaN values in the dataset.
trainp.isnull().sum().sum()

0

In [42]:
# drop the first column from train and get it into an array
x_train = np.asarray(trainp.drop('label', axis=1))  
x_test = np.asarray(testp)  

In [44]:
x_train.shape
x_test.shape

(28000, 784)

In [45]:
# preprocessing step to standardize the data
x_train = np.array(x_train, np.float32) / 255.
x_test = np.array(x_test, np.float32) / 255.

In [48]:
# change the 1D input to 2 dimensions 28 by 28
# input image dimensions
from keras import backend as K

img_rows, img_cols = 28, 28

if K.image_data_format() == 'channels_first':

    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

In [50]:
x_test.shape

(28000, 28, 28, 1)

In [56]:
num_classes = y_train.shape[1]
num_classes

10

In [110]:
# Create the network architecture
from keras.constraints import maxnorm
from keras.layers.core import Flatten, Dense, Dropout

model = Sequential()

#a rectifier activation function and a weight constraint of max norm set to 3.
model.add(Conv2D(32, (3, 3), input_shape=(28, 28, 1), padding='same', activation='relu', kernel_constraint=maxnorm(3)))

#b)	Dropout set to 20%.

model.add(Dropout(0.2))

#c)	Convolutional layer, 32 feature maps with a size of 3×3, 
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', kernel_constraint=maxnorm(3)))

#d)	Max Pool layer with size 2×2.
model.add(MaxPooling2D(pool_size=(2, 2)))

#e)	Flatten layer.
model.add(Flatten())

#f)	Fully connected layer with 512 units and a rectifier activation function.
model.add(Dense(1024, activation='relu', kernel_constraint=maxnorm(3)))

#g)	Dropout set to 50%.
model.add(Dropout(0.5))

#h)	Fully connected output layer with 10 units and a softmax activation function.
model.add(Dense(num_classes, activation='softmax'))

# Compile model
epochs = 30
lrate = 0.01

# decayed learning rate
decay = lrate/epochs

# Using stochastic gradient descent
#sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)

# categorical cross entropy for multi-classification 
model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_11 (Conv2D)           (None, 28, 28, 32)        320       
_________________________________________________________________
dropout_8 (Dropout)          (None, 28, 28, 32)        0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 28, 28, 64)        18496     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 14, 14, 64)        36928     
_________________________________________________________________
dropout_9 (Dropout)          (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 14, 14, 128)       73856     
__________

In [120]:
# Train the network
import time
start_time = time.time()

model.fit(x_train, y_train, nb_epoch=2, batch_size=32) 
#          , show_accuracy=True, verbose=1)
          
print("--- %s seconds ---" % (time.time() - start_time))

C:\Users\kmsra_000\Anaconda3\lib\site-packages\keras\models.py:848: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/2
42000/42000 [==============================] - 1078s - loss: 0.0273 - acc: 0.9920  
Epoch 2/2
42000/42000 [==============================] - 1064s - loss: 0.0254 - acc: 0.9921  
--- 2143.205869913101 seconds ---


In [121]:
sub = pd.read_csv('sample_submission.csv')


In [122]:
predictions = model.predict(x_test)

In [123]:
labels = []
for i in range(0,sub.shape[0]):
    labels.append(np.argmax(predictions[i]))

In [100]:
sub.loc[10,'Label']

0

In [124]:
for i in range(0,sub.shape[0]):
    sub.loc[i,'Label'] = labels[i]

In [125]:
sub.head()

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,0
4,5,3


In [126]:
sub.to_csv('sub6.csv', index=False)

In [ ]:
# save as JSON
json_string = model.to_json()
model.save_weights('my_model_weights.h5')